In [1]:
import random
import pylab as plt
import numpy as np

In [2]:
from matplotlib.widgets import Button
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
%matplotlib notebook

use a 2-D Gaussian function to generate the grid that will apply marching squares to:

In [3]:
def gauss_2d(mu, sigma, size=10):
  x, y = np.meshgrid(np.linspace(-1,1,size), np.linspace(-1,1,size))
  d    = np.sqrt(x*x+y*y)
  g    = np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) )
  return g

The following code constructs the March class which uses the functions you will define later on to plot an interactive visualization.

In [7]:
class March(object):
    def __init__(self,res=32,thres=0.5,size=320):

        #Initialize variables
        self.res      = res                      #Number of grid cells per axis
        self.thres    = thres                    #Threshold for binarization
        self.size     = size                     #Size of image (in pixels)
        self.contours = 0                        #Whether we're showing contours (0 = off,  1 = normal, 2 = interpolated)
        self.cmap     = self.colorMapGrayscale() #Default grayscale color map
        self.cmapi    = 0                        #Index of color map (0 = gray, 1 = plasma, 2 = custom)

        #Hardcode some cells to start with to test all cases
        self.cells    = gauss_2d(0.5,0.4,self.res)

        #Compute other useful variables from grid size
        self.step     = self.size // self.res #Spacing between grid lines (in pixels)

        #Set up axes
        self.fig, self.axes = plt.subplots()
        self.axes.set_aspect('equal')
        plt.subplots_adjust(bottom=0.2)

        #Set up buttons
        self.btog = Button(plt.axes([0.61, 0.05, 0.2, 0.075]), 'No Contours')
        self.btog.on_clicked(self.toggle_contours)
        self.bmap = Button(plt.axes([0.41, 0.05, 0.2, 0.075]), 'Grayscale')
        self.bmap.on_clicked(self.toggle_colormap)

        #Perform initial drawing
        self.redraw()

    def show(self):
        plt.show()

    def update(self):
        self.fig.canvas.draw()

    def toggle_contours(self,event):
        #Toggle whether we draw contours or not
        self.contours = (self.contours + 1) % 3
        self.redraw()

    def toggle_colormap(self,event):
        self.cmapi = (self.cmapi+1)%2
        if self.cmapi == 0:
          self.cmap = self.colorMapGrayscale()
          self.bmap.label.set_text("Grayscale")
        elif self.cmapi == 1:
          self.cmap = colorMapPlasma()
          self.bmap.label.set_text("Plasma")
        self.redraw()

    def redraw(self):
        # Regenerate a blank white canvas withou axis lines or tick marks
        self.axes.clear()
        self.axes.set_yticks([])
        self.axes.set_xticks([])
        self.axes.set_yticklabels([])
        self.axes.set_xticklabels([])

        #Invert y axis to match up with array ordering
        self.axes.invert_yaxis()

        #Draw the image from our img matrix
        self.drawImage()
        if self.contours == 0:
          for i in range(1,self.res): #Draw image grid
            self.axes.plot([0,self.size-1], [self.step*i,self.step*i], color='black', linestyle='-', linewidth=1)
            self.axes.plot([self.step*i,self.step*i], [0,self.size-1], color='black', linestyle='-', linewidth=1)
          self.btog.label.set_text('No Contours')
        else:  # Draw contours and contour grid
          for i in range(self.res): #Draw contour grid
            self.axes.plot([0,self.size-1], [self.step*(i+0.5),self.step*(i+0.5)], color='gray', linestyle='-', linewidth=1)
            self.axes.plot([self.step*(i+0.5),self.step*(i+0.5)], [0,self.size-1], color='gray', linestyle='-', linewidth=1)
          if self.contours == 1:
            self.btog.label.set_text('Rough Contours')
            self.drawTableLookupContours()
          else:
            self.btog.label.set_text('Interp. Contours')
            self.drawInterpolatedContours()

        #Update the underlying plot
        self.update()

    def colorMapGrayscale(self):
        cdict = {'red':   [[0, 0, 0],
                           [1, 1, 1]],
                 'green': [[0, 0, 0],
                           [1, 1, 1]],
                 'blue':  [[0, 0, 0],
                           [1, 1, 1]]}
        return cdict

    def drawImage(self):
        newcmp = LinearSegmentedColormap('testCmap', segmentdata=self.cmap, N=256)
        self.axes.imshow(gauss_2d(0.5,0.4,self.size),cmap=newcmp)

    def drawTableLookupContours(self):
        for y,row in enumerate(self.cells):
          for x,cell in enumerate(row):
            case = getContourCase(y,x,self.thres,self.cells)
            self.drawCellContourByCase(y,x,case)

    def drawInterpolatedContours(self):
        segments = getContourSegments(self.thres,self.cells)
        for s in segments:
          x1 = self.step*(0.5+s[0][0])
          x2 = self.step*(0.5+s[1][0])
          y1 = self.step*(0.5+s[0][1])
          y2 = self.step*(0.5+s[1][1])
          self.axes.plot([x1,x2], [y1,y2], color='green', linestyle='-', linewidth=1)

    def drawCellContourByCase(self,yrow,xcol,case):
        if case in [0,15]:
          return #Nothing to draw for empty cells, completely surrounded cells, or border cells

        #Handle saddle points
        if case in [5]:
          if disambiguateSaddle(yrow,xcol,self.thres,self.cells):
            self.drawCellContourByCase(yrow,xcol,2)
            self.drawCellContourByCase(yrow,xcol,7)
          else:
            self.drawCellContourByCase(yrow,xcol,11)
            self.drawCellContourByCase(yrow,xcol,14)
          return
        if case in [10]:
          if disambiguateSaddle(yrow,xcol,self.thres,self.cells):
            self.drawCellContourByCase(yrow,xcol,11)
            self.drawCellContourByCase(yrow,xcol,14)
          else:
            self.drawCellContourByCase(yrow,xcol,2)
            self.drawCellContourByCase(yrow,xcol,7)
          return

        #Compute coordinates based on case lookup table
        s    = self.step
        ymin = s*yrow + (0         if case in [4,6,7,8,9,11]   else s//2)
        ymax = s*yrow + (self.step if case in [1,2,6,9,13,14]  else s//2)
        xmin = s*xcol + (0         if case in [1,3,7,8,12,14]  else s//2)
        xmax = s*xcol + (self.step if case in [2,3,4,11,12,13] else s//2)
        if case in [2,7,8,13]: #Reverse direction for lines drawn up and right (i.e., x increases while y decreases)
          xmin,xmax = xmax,xmin

        #Contour lines should be drawn halfway between grid cells, so set an offset
        off = s//2
        #Smooth contours should have different color
        color = 'red' if self.contours == 1 else 'green'
        #Actually draw the contour lines
        self.axes.plot([xmin+off, xmax+off], [ymin+off, ymax+off], color=color, linestyle='-', linewidth=1)
        return

Getting the Contour Case:

In [8]:
def getContourCase(top,left,thres,cells):
    n = cells.shape[0]
    if left not in range(n-1) or top not in range(n-1):
        return 0;
    cells = np.where(cells >= thres,1,0)
    contour = np.zeros([cells.shape[0]-1,cells.shape[0]-1])
    num = cells[top][left];
    right = left+1;
    bottom = top+1;
    a = cells[top][left];
    b = cells[top][right];
    c = cells[bottom][left];
    d = cells[bottom][right];
    temp = int(str(a)+str(b)+str(d)+str(c),2)
    return temp;
    raise NotImplementedError

Disambiguating Saddle Points:

In [10]:
def disambiguateSaddle(top,left,thres,cells):
    # how to calculate the ambiguou
    n = cells.shape[0]
    if top == n-1:
        top = top-1;
    if left == n-1:
        left = left-1;
    right = left+1;
    bottom = top+1;
    a = cells[top][left];
    b = cells[top][right];
    c = cells[bottom][left];
    d = cells[bottom][right];
    return (a+b+c+d)/4 > thres
    raise NotImplementedError

Interpolating the Contour Lines:

In [11]:
def interpolate(v1,v2,t):
    if v1 == v2:
        return 0;
    return abs(t-v1)/abs(v2-v1)
    raise NotImplementedError

In [12]:
def getCellSegments(top,left,thres,cells):
    n = cells.shape[0]
    temp = []
    if top+1>n-1 or left+1>n-1:
        return [];
    a = (left + interpolate(cells[top][left], cells[top][left+1], thres), top);

    b = (left, interpolate(cells[top][left],  cells[top+1][left], thres) + top);

    c = (interpolate(cells[top+1][left], cells[top+1][left+1], thres) + left , top +1);

    d = (left+1, interpolate(cells[top][left+1], cells[top+1][left+1], thres)+top);
    newCell = np.where(cells >= thres,1,0)
    one = newCell[top][left];
    two = newCell[top][left+1];
    three = newCell[top+1][left];
    four = newCell[top+1][left+1];
    if one != two:
        temp.append(a)
    if one != three:
        temp.append(b)
    if three != four:
        temp.append(c)
    if two != four:
        temp.append(d)
    if (len(temp)==2):
        return [temp]
    elif len(temp)==4:
        if disambiguateSaddle(top,left,thres,cells)== True:
            if newCell[top][left] == 1:
                return[[b,c],[a,d]]
            else:
                return [[a,b],[c,d]]
        else:
            if newCell[top][left] == 1:
                return [[a,b],[c,d]]
            else:
                return [[b,c],[a,d]]
    elif len(temp) == 0:
        return []

In [13]:
def getContourSegments(thres,cells):
    result = [];
    n = cells.shape[0]
    for i in range(n):
        for j in range(n):
            line = getCellSegments(i,j,thres,cells)
            if len(line) == 1:
                result.append(line[0])
            elif len(line) == 2:
                result.append(line[0])
                result.append(line[1])
            else:
                continue;
    return result;
    raise NotImplementedError

Creating a Colormap:

In [14]:
def colorMapPlasma():
    cdict = {'red': [[0.0,  13, 13],
                   [0.142857143, 84, 84],
                   [0.285714286, 139, 139],
                    [0.428571428571428, 185, 185],
                    [0.571428571428571,219,219],
                    [0.714285714285714,244,244],
                    [0.857142857142857,254,254],
                    [1,240,240]],
         'green': [[0.0,  8, 8],
                   [0.142857143, 2, 2],
                   [0.285714286, 10, 10],
                    [0.428571428571428, 50, 50],
                    [0.571428571428571,92,92],
                    [0.714285714285714,136,136],
                    [0.857142857142857,188,188],
                    [1,249,249]],
         'blue':  [[0.0,  135, 135],
                   [0.142857143, 163, 163],
                   [0.285714286, 165, 165],
                    [0.428571428571428, 137, 137],
                    [0.571428571428571,104,104],
                    [0.714285714285714,73,73],
                    [0.857142857142857,43,43],
                    [1,33,33]],}
    for color in cdict.values():
        for line in color:
            line[1] = line[1]/255
            line[2]  = line[2]/255
    return cdict
    raise NotImplementedError 

In [ ]:
March(res=12,thres=0.9).show()